# What Public School characteristics point towards a "high strained system" ie high student teacher ratio?

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sqlalchemy import create_engine
from utils import db_connect

pd.set_option('display.max_columns', None)

engine = db_connect()

# bring in the data
main_df = pd.read_sql('SELECT * FROM combined_data', con=engine)

# function to mark whether a school was operational all five years of data
# creates our 'Currently operational' feature
def mark_always_operational(main_df):
    
    total_years = main_df['SURVYEAR'].nunique()
    
    main_df['SY_STATUS_TEXT'] = main_df['SY_STATUS_TEXT'].str.strip()
    
    operational_counts = (
        main_df[main_df['SY_STATUS_TEXT'] == 'Currently operational']
        .groupby('NCESSCH')['SURVYEAR']
        .nunique()
    )
    
    always_operational_schools = operational_counts[operational_counts == total_years].index
    
    main_df['concurrently_operational'] = main_df['NCESSCH'].isin(always_operational_schools)
    
    return main_df

# apply the function
mark_always_operational(main_df=main_df)

# drop records that were not fully operational across all five years
main_df = main_df[main_df['concurrently_operational'] != False]

# strip whitespace
for col in main_df.select_dtypes(include=["string"]).columns:
    main_df[col] = main_df[col].str.strip()

# begin defining data type conversion processes
# Change columns to floats
float_cols = ["X", "Y", "LATCOD", "LONCOD", "FTE", "STUTERATIO"]

# change columns to int
int_cols = [
    "OBJECTID", "GSLO", "GSHI",
    "TOTFRL", "FRELCH", "REDLCH", "DIRECTCERT",
    "PK", "KG", "G01", "G02", "G03", "G04", "G05", "G06",
    "G07", "G08", "G09", "G10", "G11", "G12", "G13",
    "UG", "AE",
    "TOTMENROL", "TOTFENROL", "TOTAL", "MEMBER",
    "AMALM", "AMALF", "AM",
    "ASALM", "ASALF", "AS",
    "BLALM", "BLALF", "BL",
    "HPALM", "HPALF", "HP",
    "HIALM", "HIALF", "HI",
    "TRALM", "TRALF", "TR",
    "WHALM", "WHALF", "WH"
]

# Change columns to strings
string_cols = [
    "NCESSCH", "SURVYEAR", "STABR", "LEAID", "ST_LEAID",
    "LEA_NAME", "SCH_NAME",
    "LSTREET1", "LSTREET2", "LCITY", "LSTATE",
    "LZIP", "LZIP4", "PHONE",
    "VIRTUAL", "SCHOOL_LEVEL", "SCHOOL_TYPE_TEXT",
    "STATUS", "SY_STATUS_TEXT", "ULOCALE", "NMCNTY",
    "CHARTER_TEXT", "LSTREET3", "TITLEI", "STITLEI", "MAGNET_TEXT"
]
 
# -1 or M -> Indicates that the data are missing.

# -2 or N -> Indicates that the data are not applicable.

# -9 -> Indicates that the data do not meet NCES data quality standards.

# function to clean NCES error codes
def clean_nces_error_codes(main_df, cols):
    error_values = ["M", "-1", "-9", "Missing", -1, -9
]
    main_df[cols] = main_df[cols].replace(error_values, np.nan)
    return main_df



# clean ALL columns 
cols = float_cols + int_cols + string_cols
main_df = clean_nces_error_codes(main_df, cols)

# convert floats safely
for col in float_cols:
    main_df[col] = pd.to_numeric(main_df[col], errors="coerce")

    # convert ints safely
for col in int_cols:
    main_df[col] = pd.to_numeric(main_df[col], errors="coerce").astype("Int64")

    # convert strings
for col in string_cols:
    main_df[col] = main_df[col].astype("string")

    # round coordinates
main_df["LATCOD"] = main_df["LATCOD"].round(4)
main_df["LONCOD"] = main_df["LONCOD"].round(4)

# extract start Year - convert to int for sorting
main_df['SURVYEAR'] = main_df['SURVYEAR'].str[:4].astype(int)

# removing virtual schools
main_df = main_df[main_df['VIRTUAL'].isin(['Not Virtual', 'Not a virtual school'])]

# drop the virtual feature
main_df.drop(columns='VIRTUAL', inplace=True)

# only keeping 'regular' public schools, removing: [ 'Career and Technical School',
# 'Special education school', 'Alternative Education School',
# 'Alternative/other school', 'Vocational school']
main_df = main_df[main_df['SCHOOL_TYPE_TEXT'].isin(['Regular school', 'Regular School'])]

# drop the SCHOOL_TYPE_TEXT feature
main_df.drop(columns='SCHOOL_TYPE_TEXT', inplace=True)

# replace na values with 0
main_df = main_df.fillna(0)

# Checking records against all five years
counts = main_df["NCESSCH"].value_counts()
keep_ids = counts[counts == 5].index
main_df = main_df[main_df["NCESSCH"].isin(keep_ids)].copy()

print(f"1. main_df shape: {main_df.shape}")

# Simplify ULOCALE
main_df["locale_category"] = main_df["ULOCALE"].str.split("-").str[1].str.split(":").str[0]

# Drop the ULOCALE feature because we now have our simplified locale_category feature
main_df.drop(columns='ULOCALE', inplace=True)

# Title I rough breakdown:

# Participating:
# 1 - Yes - School participates in Title I funding / programs
# 5 - Title I schoolwide school - ENTIRE school recieves Title I support. Funds can be used for all students
# 2 - Title I targeted assistance school - Only SPECIFIC eligible students recieve services (usually low-income or academically at risk)

# Eligible, but no program running:
# 4 - Title I schoolwide eligible school - no program - Enough low-income students to qualify for schoolwide funding, but not using it
# 1 - Title I targeted assistance eligible school - No program - Eligible for targeted assistance but not participating

# Hybrid
# 3 Title I schoolwide eligible - Title I targeted assitance program - School qualifies for schoolwide funding but has chosen to run only a targeted program

# Explicit non-participation
# 2 - No - School does not participate
# 6 - Not a Title I school

# 0
# 0 - Assuming missing, unknown, or not reported


# Conceptual differences:
# Schoolwide = whole school qualifies = High funding flexibility - Typical poverty threshold >= 40% low-income
# Targeted = only some students qualify = Limited funding flexibility - lower threshold for poverty

# standardize TITLEI
schoolwide = ['1-Yes', '5-Title I schoolwide school']
targeted = ['2-Title I targeted assistance school', '3-Title I schoolwide eligible-Title I targeted assistance program']
elig_no_participate = ['4-Title I schoolwide eligible school-No program', 
                       '1-Title I targeted assistance eligible school-No program']
not_elig = ['2-No', '6-Not a Title I school']
missing = [0]

def group_titlei(col_TITLEI):
    if col_TITLEI in missing:
        return "Unknown"
    elif col_TITLEI in schoolwide:
        return "Schoolwide"
    elif col_TITLEI in targeted:
        return "Targeted"
    elif col_TITLEI in elig_no_participate:
        return "Eligible_No_Program"
    elif col_TITLEI in not_elig:
        return "Not_Eligible"
    else:
        return "Error"
    
# apply the above function to main_df
main_df['TITLEI_GROUPED'] = main_df['TITLEI'].apply(group_titlei)

# standardize STITLEI
STITLEI_yes = ['1-Yes', 'Yes']
STITLEI_no = ['2-No', 'No']
STITLEI_unknown = [0]

def standardize_STITLEI(col_STITLEI):
    if col_STITLEI in STITLEI_yes:
        return 'Yes'
    elif col_STITLEI in STITLEI_no:
        return 'No'
    elif col_STITLEI in STITLEI_unknown:
        return 'Unknown'
    else:
        return 'Error'
    
# apply the above function to main_df    
main_df['STITLEI'] = main_df['STITLEI'].apply(standardize_STITLEI)

# update the contradticions between TITLEI and STITLEI (Updating the below to 'Targeted' group instead of 'Schoolwide')
main_df.loc[(main_df['TITLEI'] == '1-Yes') & (main_df['STITLEI'] == 'No'), 'TITLEI_GROUPED'] = 'Targeted'

# Checking records against all five years
counts = main_df["NCESSCH"].value_counts()
keep_ids = counts[counts == 5].index
main_df = main_df[main_df["NCESSCH"].isin(keep_ids)].copy()

print(f"2. main_df shape: {main_df.shape}")

# further filtering on positive student teacher ratios
main_df = main_df[main_df['STUTERATIO'] != 0.0]

# define additional redundant columns
redundant_cols = ['X', 'Y', 'OBJECTID', 'ST_LEAID', 'LSTREET1', 'LSTREET2', 'LSTREET3', 
                  'LZIP4', 'PHONE', 'AMALM', 'AMALF', 'ASALM', 'ASALF', 
                  'BLALM', 'BLALF', 'HPALM', 'HPALF', 'HIALM', 'HIALF', 'TRALM', 'TRALF', 
                  'WHALM', 'WHALF', 'STABR', 'LCITY', 'LSTATE', 'LZIP', 'SCHOOL_LEVEL', 'GSLO', 'GSHI'
                  , 'STATUS', 'SY_STATUS_TEXT', 'NMCNTY', 'DIRECTCERT', 'AE', 'TOTFENROL', 'TOTMENROL',
                  'concurrently_operational', 'TITLEI', 'STITLEI', 'MEMBER']

# drop additional redundant cols
main_df = main_df.drop(columns=redundant_cols)

# remove the large Alaska homeschool support program from data set
main_df[main_df['NCESSCH'] != '20013000253']

# trim the top percentile off
def trim_top_percentile(df, col="STUTERATIO", percentile=0.99):

    df = df.copy()

    # Calculate cutoff
    cutoff = df[col].quantile(percentile)

    # Count rows before trimming
    before_count = df.shape[0]

    # Trim
    df_trimmed = df[df[col] <= cutoff].copy()

    after_count = df_trimmed.shape[0]
    
    # updating nces error codes to No or 0 for respective columns

    print(f"{percentile*100}th percentile cutoff: {cutoff:.2f}")
    print(f"Rows before: {before_count}")
    print(f"Rows after: {after_count}")
    print(f"Rows removed: {before_count - after_count}")

    print("\nTop values after trimming:")
    print(
        df_trimmed.sort_values(col, ascending=False)[
            ["NCESSCH", "SURVYEAR", "FTE", col]
        ].head(10)
    )

    return df_trimmed

# apply the above function to main_df
main_df = trim_top_percentile(main_df, col="STUTERATIO", percentile=0.99)

# trim the bottom percentile off
def trim_bottom_percentile(df, col="STUTERATIO", percentile=0.01):

    df = df.copy()

    # Calculate cutoff
    cutoff = df[col].quantile(percentile)

    # Count rows before trimming
    before_count = df.shape[0]

    # Trim bottom values
    df_trimmed = df[df[col] >= cutoff].copy()

    after_count = df_trimmed.shape[0]

    print(f"{percentile*100}th percentile cutoff: {cutoff:.2f}")
    print(f"Rows before: {before_count}")
    print(f"Rows after: {after_count}")
    print(f"Rows removed: {before_count - after_count}")

    print("\nBottom values after trimming:")
    print(
        df_trimmed.sort_values(col, ascending=True)[
            ["NCESSCH", "SURVYEAR", "FTE", col]
        ].head(10)
    )

    return df_trimmed

main_df = trim_bottom_percentile(main_df)

# create our high-strain feature
main_df["high_strain"] = (main_df["STUTERATIO"] > 20).astype(int)

# keeping all records with 5 years of data
counts = main_df["NCESSCH"].value_counts()
keep_ids = counts[counts == 5].index
main_df = main_df[main_df["NCESSCH"].isin(keep_ids)].copy()

print(f"3. main_df shape: {main_df.shape}")

Connection successful
1. main_df shape: (202600, 80)
2. main_df shape: (202600, 81)
99.0th percentile cutoff: 28.46
Rows before: 198008
Rows after: 196028
Rows removed: 1980

Top values after trimming:
             NCESSCH  SURVYEAR        FTE  STUTERATIO
387512  390437500293      2017  13.000000       28.46
346102  320006000591      2020  56.000000       28.46
387772  390461104403      2017  17.500000       28.46
370771  350165001122      2020   9.700000       28.45
23232   390468902689      2022  18.540001       28.43
35899   470014802171      2022  15.830000       28.43
214563  120165001822      2021   7.000000       28.43
145306  370504002912      2021   7.530000       28.42
306215  271824000964      2020  26.209999       28.42
347309  320006000730      2020  62.000000       28.42
1.0th percentile cutoff: 6.00
Rows before: 196028
Rows after: 194103
Rows removed: 1925

Bottom values after trimming:
             NCESSCH  SURVYEAR    FTE  STUTERATIO
310838  290000901352      2018   2.

In [2]:
# updating nces error codes to No or 0 for respective columns
values = ["N", "-2","Not applicable", "Not Applicable",-2,]
main_df.loc[main_df['CHARTER_TEXT'].isin(values), 'CHARTER_TEXT'] = 'No'
main_df.loc[main_df['MAGNET_TEXT'].isin(values), 'MAGNET_TEXT'] = 'No'
main_df.loc[main_df['FRELCH'].isin(values), 'FRELCH'] = 0
main_df.loc[main_df['REDLCH'].isin(values), 'REDLCH'] = 0

In [3]:
main_df = main_df.sort_values(['NCESSCH','SURVYEAR'])

In [4]:
main_df['enrollment_growth'] = (
    main_df.groupby('NCESSCH')['TOTAL']
    .pct_change()
)

In [6]:
main_df['fte_growth'] = (
    main_df.groupby('NCESSCH')['FTE']
    .pct_change()
)


In [7]:
main_df['staffing_gap'] = main_df['enrollment_growth'] - main_df['fte_growth']


In [8]:
main_df['previous_strain'] = (
    main_df.groupby('NCESSCH')['high_strain']
    .shift(1)
)


In [10]:
main_df['enrollment_volatility'] = (
    main_df.groupby('NCESSCH')['TOTAL']
    .rolling(3)
    .std()
    .reset_index(level=0, drop=True)
)



In [11]:
main_df['poverty_rate'] = main_df['TOTFRL'] / main_df['TOTAL']


In [ ]:
main_df['avg_poverty_3yr'] = (
    main_df.groupby('NCESSCH')['poverty_rate']
    .rolling(3)
    .mean()
    .reset_index(level=0, drop=True)
)

# What predicts entering strain or what predicts sustained strain?
# Do schools adjust staffing after growth
# What factors predict shools entering high student-teach strain

In [15]:
model_df = main_df.dropna(subset=['previous_strain'])


In [18]:
model_df = main_df.copy()

model_df = model_df.dropna(subset=[
    'previous_strain',
    'enrollment_growth',
    'fte_growth',
    'staffing_gap'
])


In [20]:
model_df[['enrollment_growth','fte_growth','staffing_gap']].describe()


,enrollment_growth,fte_growth,staffing_gap
count,147524.0,147524.000000,147524.0
mean,-0.006706,0.005956,-0.012662
std,0.134072,0.138126,0.138517
min,-0.874459,-0.803279,-2.597158
25%,-0.056338,-0.053319,-0.072331
50%,-0.008152,0.000000,-0.008513
75%,0.036683,0.051016,0.05284
max,24.0,9.000000,15.0


In [22]:
main_df = main_df.sort_values(['NCESSCH', 'SURVYEAR'])
main_df['SURVYEAR'] = main_df['SURVYEAR'].astype(int)

main_df['entered_strain'] = (
    (main_df['previous_strain'] == 0) &
    (main_df['high_strain'] == 1)
).astype(int)


In [23]:
model_df = main_df.dropna(subset=['previous_strain'])


In [25]:
model_df['entered_strain'].value_counts(normalize=True)


entered_strain
0    0.970812
1    0.029188
Name: proportion, dtype: float64

In [27]:
model_df = main_df.dropna(subset=['previous_strain']).copy()


In [28]:
features = [
    'enrollment_growth',
    'fte_growth',
    'staffing_gap',
    'poverty_rate',
    'previous_strain'
]

X = model_df[features]
y = model_df['entered_strain']


In [29]:
X = X.fillna(0)


In [30]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


In [31]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=300,
    min_samples_split=5,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

rf.fit(X_train, y_train)


,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",300
,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.Note: This parameter is tree-specific.",'gini'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",5
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=""sqrt""The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to `""sqrt""`.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",'sqrt'
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current node, ``N_t_L`` is the number of samples in theleft child, and ``N_t_R`` is the number of samples in the right child.``N``, ``N_t``, ``N_t_R`` and ``N_t_L`` all refer to the weighted sum,if ``sample_weight`` is passed... versionadded:: 0.19",0.0
,"bootstrap bootstrap: bool, default=TrueWhether bootstrap samples are used when building trees. If False, thewhole dataset is used to build each tree.",True
,"oob_score oob_score: bool or callable, default=FalseWhether to use out-of-bag samples to estimate the generalization score.By default, :func:`~sklearn.metrics.accuracy_score` is used.Provide a callable with signature `metric

In [32]:
from sklearn.metrics import roc_auc_score

pred_probs = rf.predict_proba(X_test)[:,1]
auc = roc_auc_score(y_test, pred_probs)

print("AUC:", round(auc,3))


AUC: 0.863


In [33]:
from sklearn.metrics import classification_report

preds = rf.predict(X_test)
print(classification_report(y_test, preds))


              precision    recall  f1-score   support

           0       0.97      1.00      0.98     28644
           1       0.28      0.05      0.08       861

    accuracy                           0.97     29505
   macro avg       0.62      0.52      0.53     29505
weighted avg       0.95      0.97      0.96     29505



In [34]:
import pandas as pd

importances = pd.Series(
    rf.feature_importances_,
    index=features
).sort_values(ascending=False)

print(importances)


staffing_gap         0.490174
fte_growth           0.207396
enrollment_growth    0.164843
poverty_rate         0.113971
previous_strain      0.023616
dtype: float64


In [37]:
main_df = main_df.sort_values(['NCESSCH', 'SURVYEAR'])
main_df.head()


,NCESSCH,SURVYEAR,LEAID,LEA_NAME,SCH_NAME,CHARTER_TEXT,MAGNET_TEXT,TOTFRL,FRELCH,REDLCH,PK,KG,G01,G02,G03,G04,G05,G06,G07,G08,G09,G10,G11,G12,G13,UG,TOTAL,FTE,STUTERATIO,AM,AS,BL,HP,HI,TR,WH,LATCOD,LONCOD,locale_category,TITLEI_GROUPED,high_strain,enrollment_growth,fte_growth,staffing_gap,previous_strain,enrollment_volatility,poverty_rate,avg_poverty_3yr,entered_strain
457914,100000400012,2017,1000004,Charter School of Wilmington,Charter School of Wilmington,Yes,No,0,0,0,0,0,0,0,0,0,0,0,0,0,248,251,239,233,0,0,971,51.0,19.04,5,327,71,1,46,19,502,39.7536,-75.5875,City,Unknown,0,<NA>,NaN,<NA>,NaN,NaN,0.0,NaN,0
280388,100000400012,2018,1000004,Charter School of Wilmington,Charter School of Wilmington,Yes,No,0,0,0,0,0,0,0,0,0,0,0,0,0,241,248,249,234,0,0,972,51.0,19.10,6,311,76,0,46,23,510,39.7536,-75.5875,City,Schoolwide,0,0.00103,0.000000,0.00103,0.0,NaN,0.0,NaN,0
127803,100000400012,2020,1000004,Charter School of Wilmington,Charter School of Wilmington,Yes,No,0,0,0,0,0,0,0,0,0,0,0,0,0,248,242,240,235,0,0,965,49.0,19.69,4,286,71,0,65,35,504,39.7536,-75.5875,City,Schoolwide,0,-0.007202,-0.039216,0.032014,0.0,3.785939,0.0,0.0,0
209608,100000400012,2021,1000004,Charter School of Wilmington,Charter School of Wilmington,Yes,No,0,0,0,0,0,0,0,0,0,0,0,0,0,243,252,238,238,0,0,971,50.0,19.42,2,285,85,0,72,34,493,39.7536,-75.5875,City,Schoolwide,0,0.006218,0.020408,-0.014191,0.0,3.785939,0.0,0.0,0
85463,100000400012,2022,1000004,Charter School of Wilmington,Charter School of Wilmington,Yes,0,0,0,0,0,0,0,0,0,0,0,0,0,0,242,243,245,238,0,0,968,51.0,18.98,4,306,93,1,76,40,448,39.7536,-75.5875,City,Unknown,0,-0.00309,0.020000,-0.02309,0.0,3.000000,0.0,0.0,0


In [38]:
main_df['SURVYEAR'] = main_df['SURVYEAR'].astype(int)


In [39]:
main_df['strain_rolling_3yr'] = (
    main_df.groupby('NCESSCH')['high_strain']
    .rolling(3)
    .sum()
    .reset_index(level=0, drop=True)
)


In [40]:
main_df['chronic_strain'] = (
    main_df['strain_rolling_3yr'] >= 3
).astype(int)



In [41]:
chronic_schools = (
    main_df.groupby('NCESSCH')['chronic_strain']
    .max()
)

chronic_schools.value_counts(normalize=True)


chronic_strain
0    0.948185
1    0.051815
Name: proportion, dtype: float64

NameError: name 'school_chronic' is not defined